In [1]:
import logging
import os
from pathlib import Path

import joblib
import mlflow
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Cấu hình MLflow Tracking Server
# Đảm bảo server đang chạy tại địa chỉ này (mlflow ui --port 5001)
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/") 

# Cấu hình logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s - %(message)s",
)
logger = logging.getLogger("housing")


def train():
    """
    Tải dữ liệu, huấn luyện mô hình SGDRegressor, đánh giá và log kết quả
    và mô hình vào MLflow Tracking Server.
    """
    
    # 1. Cấu hình Đường dẫn
    mlflow.set_experiment("housing_price_training")
    
    # PROJECT_ROOT sẽ là thư mục D:\Boothcamp_Mlops nếu bạn chạy từ đó
    PROJECT_ROOT = Path(os.getcwd())
    DATA_PATH = PROJECT_ROOT / "data" / "housing.csv"
    
    # SỬA: Thay đổi ARTIFACT_DIR để lưu trữ ở thư mục 'artifacts' cấp cao nhất
    ARTIFACT_DIR = PROJECT_ROOT / "artifacts"
    ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)
    MODEL_PATH = ARTIFACT_DIR / "housing_linear.joblib"

    logger.info(f"Data path: {DATA_PATH}")
    logger.info(f"Artifact dir: {ARTIFACT_DIR}")

    # 2. Tải và Chuẩn bị Dữ liệu
    if not DATA_PATH.exists():
        logger.error(f"Dữ liệu không tìm thấy tại: {DATA_PATH}")
        return # Dừng nếu không tìm thấy dữ liệu

    logger.info("Loading dataset...")
    df = pd.read_csv(DATA_PATH)
    logger.info(f"Loaded {len(df)} rows and {len(df.columns)} columns")

    logger.info("Preparing features and target...")
    TARGET = "Price"
    NUM_FEATURES = [
        "Avg. Area Income",
        "Avg. Area House Age",
        "Avg. Area Number of Rooms",
        "Avg. Area Number of Bedrooms",
        "Area Population",
    ]

    X = df[NUM_FEATURES]
    y = df[TARGET]

    logger.info("Splitting train/test...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # 3. Xây dựng Pipeline
    logger.info("Building pipeline...")
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), NUM_FEATURES),
            # Thêm các bước xử lý dữ liệu khác (như OneHotEncoder cho CAT_FEATURES) tại đây nếu cần
        ],
        remainder="drop",
    )
    
    # Tham số mô hình
    max_iter = 5000
    tol = 1e-3
    learning_rate = "optimal"
    random_state = 20
    
    # Nhóm các tham số để log dễ dàng hơn
    hyperparams = {
        "max_iter": max_iter,
        "tol": tol,
        "learning_rate": learning_rate,
        "random_state": random_state,
        "model_name": "SGDRegressor"
    }

    model = Pipeline(
        steps=[
            ("preprocess", preprocessor),
            (
                "regressor",
                SGDRegressor(
                    max_iter=max_iter,
                    tol=tol,
                    learning_rate=learning_rate,
                    random_state=random_state,
                    verbose=1, # Giữ lại để xem quá trình fit
                ),
            ),
        ]
    )

    # 4. Huấn luyện và Log với MLflow
    logger.info("Training model...")
    # Tên run nên bao gồm các tham số chính để dễ phân biệt
    run_name = f"SGD_{learning_rate}_iter{max_iter}"
    
    with mlflow.start_run(run_name=run_name) as run:
        
        # SỬA: Log Hyperparameters bằng mlflow.log_dict
        mlflow.log_dict(hyperparams, "model_params.json")
        logger.info(f"Logged hyperparameters: {hyperparams}")
        
        # Huấn luyện mô hình
        model.fit(X_train, y_train)

        # 5. Đánh giá Mô hình
        logger.info("Evaluating model performance...")
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        # Tính toán Metrics
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_mae = mean_absolute_error(y_train, y_train_pred)
        train_r2 = r2_score(y_train, y_train_pred)

        test_mse = mean_squared_error(y_test, y_test_pred)
        test_mae = mean_absolute_error(y_test, y_test_pred)
        test_r2 = r2_score(y_test, y_test_pred)

        # In và Log Metrics
        metrics = {
            "train_mse": train_mse, "train_mae": train_mae, "train_r2": train_r2,
            "test_mse": test_mse, "test_mae": test_mae, "test_r2": test_r2,
            "train_rmse": train_mse**0.5, "test_rmse": test_mse**0.5,
        }
        
        logger.info("Model performance metrics:")
        for key, value in metrics.items():
            mlflow.log_metric(key, value)
            logger.info(f"  {key}: {value:.4f}")
        
        # SỬA: Loại bỏ phần log metrics cũ trùng lặp
        
        # 6. Lưu trữ Model
        
        # Cách 1: Lưu bằng joblib (cách truyền thống)
        joblib.dump(model, MODEL_PATH)
        mlflow.log_artifact(MODEL_PATH, "artifacts")
        logger.info(f"Model saved locally and logged as artifact: {MODEL_PATH}")
        
        # Cách 2: Lưu bằng mlflow.sklearn (cách khuyến nghị)
        mlflow.sklearn.log_model(model, "sklearn_model")
        logger.info("Model logged using mlflow.sklearn.log_model")
        
    logger.info(f"\n✅ Hoàn thành Run ID: {run.info.run_id} trong Experiment: {mlflow.get_experiment(run.info.experiment_id).name}")


if __name__ == "__main__":
    train()


2025/11/01 17:22:01 INFO mlflow.tracking.fluent: Experiment with name 'housing_price_training' does not exist. Creating a new experiment.
2025-11-01 17:22:02,059 INFO housing - Data path: d:\Boothcamp_Mlops\data\housing.csv
2025-11-01 17:22:02,060 INFO housing - Artifact dir: d:\Boothcamp_Mlops\artifacts
2025-11-01 17:22:02,061 INFO housing - Loading dataset...
2025-11-01 17:22:02,108 INFO housing - Loaded 5000 rows and 7 columns
2025-11-01 17:22:02,111 INFO housing - Preparing features and target...
2025-11-01 17:22:02,145 INFO housing - Splitting train/test...
2025-11-01 17:22:02,159 INFO housing - Building pipeline...
2025-11-01 17:22:02,161 INFO housing - Training model...
2025-11-01 17:22:02,815 INFO housing - Logged hyperparameters: {'max_iter': 5000, 'tol': 0.001, 'learning_rate': 'optimal', 'random_state': 20, 'model_name': 'SGDRegressor'}
2025-11-01 17:22:02,859 INFO housing - Evaluating model performance...
2025-11-01 17:22:02,877 INFO housing - Model performance metrics:
202

-- Epoch 1
Norm: 5819269802357.51, NNZs: 5, Bias: -284411575081.759277, T: 4000, Avg. loss: 133507662928172757676982272.000000
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 4101317096933.26, NNZs: 5, Bias: 2646851901885.573730, T: 8000, Avg. loss: 13719414432743692057968640.000000
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 2803091923143.66, NNZs: 5, Bias: -83373759522.158325, T: 12000, Avg. loss: 4560999936234764808224768.000000
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 2854502212035.48, NNZs: 5, Bias: -440619445678.418335, T: 16000, Avg. loss: 2136594426981578010787840.000000
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 1607096821261.89, NNZs: 5, Bias: -818641017402.999146, T: 20000, Avg. loss: 1140891627127447455006720.000000
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 902361242653.86, NNZs: 5, Bias: 605603697280.083252, T: 24000, Avg. loss: 633722000965620960591872.000000
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 987329948470.51, NNZ

2025-11-01 17:22:03,050 INFO housing -   test_rmse: 106634.1667
2025-11-01 17:22:03,212 INFO housing - Model saved locally and logged as artifact: d:\Boothcamp_Mlops\artifacts\housing_linear.joblib
2025/11/01 17:22:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/01 17:22:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-11-01 17:22:12,076 INFO housing - Model logged using mlflow.sklearn.log_model
2025-11-01 17:22:12,419 INFO housing - 
✅ Hoàn thành Run ID: d9949e29a3eb42f1aafa76c5db9dd105 trong Experiment: housing_price_training


🏃 View run SGD_optimal_iter5000 at: http://127.0.0.1:5000/#/experiments/856326979871672178/runs/d9949e29a3eb42f1aafa76c5db9dd105
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/856326979871672178


In [ ]:
from enum import Enum

import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

from scripts.session_3.router import predict, utils

app = FastAPI()
app.include_router(predict.housing_router)
app.include_router(utils.utils_router)


@app.get("/")
def root():
    return {"message": "Hello, FastAPI!"}


class Method(str, Enum):
    add = "add"
    subtract = "subtract"
    multiply = "multiply"
    divide = "divide"


class CalculateRequest(BaseModel):
    method: Method
    num1: float
    num2: float


class CalculateResponse(BaseModel):
    result: float


@app.post("/calculate", response_model=CalculateResponse)
def calculate(request: CalculateRequest) -> CalculateResponse:
    if request.method == Method.add:
        result = request.num1 + request.num2
    elif request.method == Method.subtract:
        result = request.num1 - request.num2
    elif request.method == Method.multiply:
        result = request.num1 * request.num2
    elif request.method == Method.divide:
        result = request.num1 / request.num2
    else:
        raise ValueError(f"Invalid method: {request.method}")
    return CalculateResponse(result=result)


if __name__ == "__main__":
    uvicorn.run("scripts.session_3.api:app", host="0.0.0.0", port=3000, reload=True)


INFO:     Will watch for changes in these directories: ['d:\\Boothcamp_Mlops']
INFO:     Uvicorn running on http://0.0.0.0:3000 (Press CTRL+C to quit)
INFO:     Started reloader process [7348] using StatReload
